In [1]:
import sys
print(sys.executable)
print(sys.version)
!{sys.executable} -m pip install pandas pyarrow

c:\Users\Manny\AppData\Local\Programs\Python\Python312\python.exe
3.12.0 (tags/v3.12.0:0fb18b0, Oct  2 2023, 13:03:39) [MSC v.1935 64 bit (AMD64)]



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import pyarrow
from pathlib import Path

project_dir = Path("C:/Users/Manny/Desktop/aidms/aidms-final-project").resolve()
import sys
if str(project_dir) not in sys.path:
    sys.path.append(str(project_dir))

from loadOEDIData import load_all, load_timeseries_for_buildings

all_data = load_all()
ts_index = all_data["timeseries_index"]
sample_ts = load_timeseries_for_buildings(ts_index, n_files=1)
sample_ts.head()


=== Loading OEDI dataset pieces ===
[load_metadata] Reading upgrade0.parquet ...
[load_metadata] Combined metadata shape: (549971, 771)
[load_dictionaries] Reading data_dictionary.tsv ...
[load_dictionaries] data_dictionary shape: (841, 5)
[load_dictionaries] Reading enumeration_dictionary.tsv ...
[load_dictionaries] UTF-8 decode failed, retrying with latin-1 ...
[load_dictionaries] enumeration_dictionary shape: (58457, 3)
[build_timeseries_index] Found timeseries files: 1000
[load_state_aggregates] Reading up00-ma-mobile_home.csv ...
[load_state_aggregates] Reading up00-ma-multi-family_with_2_-_4_units.csv ...
[load_state_aggregates] Reading up00-ma-multi-family_with_5plus_units.csv ...
[load_state_aggregates] Reading up00-ma-single-family_attached.csv ...
[load_state_aggregates] Reading up00-ma-single-family_detached.csv ...
[load_state_aggregates] Combined aggregate shape: (175200, 97)
=== Done loading OEDI dataset pieces ===
[load_timeseries_for_buildings] Reading 100043-0.parquet 

,bldg_id,timestamp,in.sqft,out.electricity.ceiling_fan.energy_consumption..kwh,out.electricity.clothes_dryer.energy_consumption..kwh,out.electricity.clothes_washer.energy_consumption..kwh,out.electricity.cooling.energy_consumption..kwh,out.electricity.cooling_fans_pumps.energy_consumption..kwh,out.electricity.dishwasher.energy_consumption..kwh,out.electricity.ev_charging.energy_consumption..kwh,...,out.schedules.no_space_heating,out.schedules.occupants,out.schedules.peak_period,out.schedules.plug_loads_other,out.schedules.plug_loads_tv,out.schedules.power_outage,out.schedules.pre_peak_period,out.schedules.vacancy,building_id,upgrade_id
0,100043,2018-01-01 00:15:00,1228.0,0.00174,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,NaN,0.578,0.0389,0.0,NaN,0.0,100043,0
1,100043,2018-01-01 00:30:00,1228.0,0.00177,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,NaN,0.576,0.0360,0.0,NaN,0.0,100043,0
2,100043,2018-01-01 00:45:00,1228.0,0.00195,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,NaN,0.562,0.0159,0.0,NaN,0.0,100043,0
3,100043,2018-01-01 01:00:00,1228.0,0.00195,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,NaN,0.562,0.0159,0.0,NaN,0.0,100043,0
4,100043,2018-01-01 01:15:00,1228.0,0.00195,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,NaN,0.562,0.0159,0.0,NaN,0.0,100043,0


In [3]:
from loadOEDIData import load_all, load_timeseries_for_buildings
import pandas as pd

# Load everything
all_data = load_all()
meta = all_data["metadata"]
ts_index = all_data["timeseries_index"]
agg_ma = all_data["state_aggregates"]

# Restrict metadata to just the buildings we actually downloaded timeseries for
bldg_ids = ts_index["building_id"].unique()
meta_sample = meta[meta["bldg_id"].isin(bldg_ids)].copy()

print("Timeseries buildings:", len(bldg_ids))
print("Metadata sample rows:", len(meta_sample))

# Peek at some key columns if they exist
candidate_cols = [
    "bldg_id",
    "in.county_name",
    "in.ahs_region",
    "in.representative_income",
    "in.building_type_recs",
    "in.tenure",                       # owner vs renter
    "in.utilityrate_electricity",      # sometimes there are utility rate fields
]

existing_cols = [c for c in candidate_cols if c in meta_sample.columns]
meta_sample[existing_cols].head()


=== Loading OEDI dataset pieces ===
[load_metadata] Reading upgrade0.parquet ...
[load_metadata] Combined metadata shape: (549971, 771)
[load_dictionaries] Reading data_dictionary.tsv ...
[load_dictionaries] data_dictionary shape: (841, 5)
[load_dictionaries] Reading enumeration_dictionary.tsv ...
[load_dictionaries] UTF-8 decode failed, retrying with latin-1 ...
[load_dictionaries] enumeration_dictionary shape: (58457, 3)
[build_timeseries_index] Found timeseries files: 1000
[load_state_aggregates] Reading up00-ma-mobile_home.csv ...
[load_state_aggregates] Reading up00-ma-multi-family_with_2_-_4_units.csv ...
[load_state_aggregates] Reading up00-ma-multi-family_with_5plus_units.csv ...
[load_state_aggregates] Reading up00-ma-single-family_attached.csv ...
[load_state_aggregates] Reading up00-ma-single-family_detached.csv ...
[load_state_aggregates] Combined aggregate shape: (175200, 97)
=== Done loading OEDI dataset pieces ===
Timeseries buildings: 1000
Metadata sample rows: 1000


,bldg_id,in.county_name,in.ahs_region,in.representative_income,in.tenure
123,124,Middlesex County,"CBSA Boston-Cambridge-Newton, MA-NH",NaN,Not Available
141,142,Middlesex County,"CBSA Boston-Cambridge-Newton, MA-NH",108302.0,Owner
1021,1022,Berkshire County,Non-CBSA New England,92529.0,Owner
1080,1081,Worcester County,Non-CBSA New England,151522.0,Owner
1120,1121,Berkshire County,Non-CBSA New England,28284.0,Owner


In [4]:
# Load timeseries for, say, 50 buildings (so it runs fast)
ts_sample = load_timeseries_for_buildings(ts_index, n_files=50)

print("ts_sample shape:", ts_sample.shape)
print("ts_sample columns (first 15):")
print(list(ts_sample.columns[:15]))

# All electricity end-use columns
elec_cols = [
    c for c in ts_sample.columns
    if c.startswith("out.electricity.") and c.endswith(".energy_consumption..kwh")
]
print("Number of electricity end-use columns:", len(elec_cols))

# Total whole-house electricity kWh in each 15-min step
ts_sample["kwh_total"] = ts_sample[elec_cols].sum(axis=1)

ts_sample[["bldg_id", "timestamp", "kwh_total"]].head()


[load_timeseries_for_buildings] Reading 100043-0.parquet ...
[load_timeseries_for_buildings] Reading 10010-0.parquet ...
[load_timeseries_for_buildings] Reading 100158-0.parquet ...
[load_timeseries_for_buildings] Reading 100185-0.parquet ...
[load_timeseries_for_buildings] Reading 100194-0.parquet ...
[load_timeseries_for_buildings] Reading 100204-0.parquet ...
[load_timeseries_for_buildings] Reading 100208-0.parquet ...
[load_timeseries_for_buildings] Reading 100325-0.parquet ...
[load_timeseries_for_buildings] Reading 100339-0.parquet ...
[load_timeseries_for_buildings] Reading 100440-0.parquet ...
[load_timeseries_for_buildings] Reading 100500-0.parquet ...
[load_timeseries_for_buildings] Reading 100512-0.parquet ...
[load_timeseries_for_buildings] Reading 100538-0.parquet ...
[load_timeseries_for_buildings] Reading 100602-0.parquet ...
[load_timeseries_for_buildings] Reading 100649-0.parquet ...
[load_timeseries_for_buildings] Reading 10069-0.parquet ...
[load_timeseries_for_build

,bldg_id,timestamp,kwh_total
0,100043,2018-01-01 00:15:00,0.93137
1,100043,2018-01-01 00:30:00,0.94122
2,100043,2018-01-01 00:45:00,0.92111
3,100043,2018-01-01 01:00:00,0.93854
4,100043,2018-01-01 01:15:00,0.91068


In [5]:
# Annual kWh per building from ts_sample
annual_kwh = (
    ts_sample.groupby("bldg_id")["kwh_total"]
    .sum()
    .reset_index(name="annual_kwh")
)

annual_kwh.describe()

,bldg_id,annual_kwh
count,50.000000,50.000000
mean,93876.140000,26298.279297
std,24966.516152,21581.371094
min,10010.000000,1079.192017
25%,100455.000000,13500.969727
50%,101244.500000,21114.399414
75%,101676.000000,34025.972168
max,102103.000000,121652.367188


In [6]:
# Join to metadata subset
meta_for_sample = meta_sample.merge(
    annual_kwh, left_on="bldg_id", right_on="bldg_id", how="inner"
)

# Example: check distribution by income band and building type (if available)
for col in ["in.representative_income", "in.building_type_recs"]:
    if col in meta_for_sample.columns:
        print(f"\nValue counts for {col}:")
        print(meta_for_sample[col].value_counts().head())


Value counts for in.representative_income:
in.representative_income
38644.0     2
190819.0    1
73823.0     1
111397.0    1
134088.0    1
Name: count, dtype: int64


In [7]:
geo_cols = [c for c in ["in.county_name", "in.ahs_region"] if c in meta_for_sample.columns]
meta_for_sample[geo_cols].value_counts().head(20)

in.county_name     in.ahs_region                      
Middlesex County   CBSA Boston-Cambridge-Newton, MA-NH    11
Worcester County   Non-CBSA New England                    7
Essex County       CBSA Boston-Cambridge-Newton, MA-NH     5
Norfolk County     CBSA Boston-Cambridge-Newton, MA-NH     5
Plymouth County    CBSA Boston-Cambridge-Newton, MA-NH     5
Hampshire County   Non-CBSA New England                    4
Suffolk County     CBSA Boston-Cambridge-Newton, MA-NH     4
Barnstable County  Non-CBSA New England                    2
Berkshire County   Non-CBSA New England                    2
Bristol County     Non-CBSA New England                    2
Hampden County     Non-CBSA New England                    2
Dukes County       Non-CBSA New England                    1
Name: count, dtype: int64